# **STOKES ELEMENT TESTING**

## **Imports**

In [25]:
try:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-release-real.sh" -O "/tmp/firedrake-install.sh"
    !bash "/tmp/firedrake-install.sh"
    from firedrake import *  # noqa: F401
except:
    from firedrake import *  # noqa: F401

--2025-12-31 17:23:27--  https://fem-on-colab.github.io/releases/firedrake-install-release-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4767 (4.7K) [application/x-sh]
Saving to: ‘/tmp/firedrake-install.sh’

/tmp/firedrake-inst 100%[===================>]   4.66K  --.-KB/s    in 0s      

2025-12-31 17:23:28 (45.9 MB/s) - ‘/tmp/firedrake-install.sh’ saved [4767/4767]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FIREDRAKE_INSTALLED=/usr/local/share/fem-on-colab/firedrake.installed
+ [[ ! -f /usr/local/share/fem-on-colab/firedrake.installed ]]
+ set +x
























#############################################################

# **Code**

In [26]:
# 1. Mesh Definition
logN = 8

# Unit square mesh
# N = round(2**logN)
# mesh = UnitSquareMesh(N, N)

# L-shape
coords = np.array([
    [0.0, 0.0],  [1.0, 0.0],  [1.0, 1.0],   [0.0, 1.0],
    [-1.0, 1.0], [-1.0, 0.0], [-1.0, -1.0], [0.0, -1.0]
], dtype=float)
cells = np.array([
    [0, 1, 2], [0, 2, 3],     # top right
    [0, 3, 4], [0, 4, 5],     # top left
    [0, 5, 6], [0, 6, 7],     # bottom left
], dtype=np.int32)
plex = PETSc.DMPlex().createFromCellList(
    2, cells, coords, interpolate=True, comm=PETSc.COMM_WORLD
)
mesh = Mesh(plex)
mesh = MeshHierarchy(mesh, logN-1)[-1]

In [27]:
# 2. Function Spaces
V = FunctionSpace(mesh, "CG", 2)

# 3. Trial and Test Functions
psi = TrialFunction(V)
phi = TestFunction(V)

u = curl(psi)
v = curl(phi)

In [ ]:
# 4. Variational Form
n = FacetNormal(mesh)
h = CellDiameter(mesh)
sigma = Constant(20.0)  # Penalty parameter

def a_biharmonic(u, v, ds_degree=None, penalty_only=False):
    dev = lambda u_ref : sym(grad(u_ref))
    jump_dev = lambda u_ref : 2 * avg(sym(outer(u_ref, n)))

    ds_ = ds if (ds_degree is None) else ds(degree=ds_degree)
    dS_ = dS if (ds_degree is None) else dS(degree=ds_degree)

    if penalty_only:
        term = 2 * inner(jump_dev(u), jump_dev(v)) * dS_
        term += 2 * inner(sym(outer(u, n)), sym(outer(v, n))) * ds_
    else:
        # Volume term: 2 * dev(u) : dev(v)
        term = 2 * inner(dev(u), dev(v)) * dx

        # Interior Facets
        term -= 2 * inner(avg(dev(u)), jump_dev(v)) * dS_
        term -= 2 * inner(avg(dev(v)), jump_dev(u)) * dS_
        term += 2 * (sigma / avg(h)) * inner(jump_dev(u), jump_dev(v)) * dS_

        # Boundary Facets (Weak imposition of no-slip BC u=0)
        term -= 2 * inner(1/2 * dev(u), sym(outer(v, n))) * ds_
        term -= 2 * inner(1/2 * dev(v), sym(outer(u, n))) * ds_
        term += 2 * (sigma / h) * inner(sym(outer(u, n)), sym(outer(v, n))) * ds_

    return term

A = a_biharmonic(u, v)
M = inner(u, v) * dx

In [ ]:
# 5. Solver
# Boundary conditions: psi = 0 on boundary (u.n = 0)
bcs = [DirichletBC(V, 0, "on_boundary")]

eigenproblem = LinearEigenproblem(A, M, bcs=bcs)

# Request 6 eigenvalues
n_evals = 6
solver = LinearEigensolver(eigenproblem, n_evals, solver_parameters={
    "eps_type": "krylovschur",
    "eps_target": 0.0,
    "eps_which": "TARGET_MAGNITUDE",
    "st_type": "sinvert",
    "st_pc_type": "lu",
    "st_pc_factor_mat_solver_type": "mumps"
})

n_found = solver.solve()
print(f"Found {n_found} eigenvalues.")

Found 7 eigenvalues.


In [ ]:
# 6. Output
for i in range(n_found):
    lam = solver.eigenvalue(i)
    print(f"Eigenvalue {i}: {lam}")

# # Save the first eigenmode
# # The solver returns the streamfunction psi
# psi_mode_real, psi_mode_imag = solver.eigenfunction(0)
# psi_mode_real.rename("Streamfunction")

# # Compute velocity u = curl(psi)
# # In 2D, curl(psi) = (d(psi)/dy, -d(psi)/dx)
# u_mode = project(curl(psi_mode_real), VectorFunctionSpace(mesh, "CG", 1))
# u_mode.rename("Velocity")

# outfile = File("stokes_streamfunction.pvd")
# outfile.write(psi_mode_real, u_mode)
# print("Saved first eigenmode to stokes_streamfunction.pvd")

Eigenvalue 0: (32.476346293358795+0j)
Eigenvalue 1: (37.140986567145184+0j)
Eigenvalue 2: (42.2113149327788+0j)
Eigenvalue 3: (49.378293754419126+0j)
Eigenvalue 4: (56.10152423973187+0j)
Eigenvalue 5: (70.31351651097286+0j)
Eigenvalue 6: (71.18140792299286+0j)
